In [5]:
import os
os.environ['USE_PYGEOS'] = '0'
os.environ['PROJ_LIB'] = '/opt/conda/share/proj/'
n_cpus = os.cpu_count()
n_threads = n_cpus
os.environ['OMPI_MCA_rmaps_base_oversubscribe'] = '1'
os.environ['USE_PYGEOS'] = '0'
os.environ['PROJ_LIB'] = '/opt/conda/share/proj/'
os.environ['NUMEXPR_MAX_THREADS'] = f'{n_threads}'
os.environ['NUMEXPR_NUM_THREADS'] = f'{n_threads}'
os.environ['OMP_THREAD_LIMIT'] = f'{n_threads}'
os.environ["OPENBLAS_NUM_THREADS"] = f'{n_threads}' 
os.environ["MKL_NUM_THREADS"] = f'{n_threads}'
os.environ["VECLIB_MAXIMUM_THREADS"] = f'{n_threads}'
os.environ["OMP_DYNAMIC"] = f'TRUE'
os.environ['TREELITE_BIND_THREADS'] = '0'
os.environ["OMP_NUM_THREADS"] = f'{n_threads}'
os.environ["OPENBLAS_NUM_THREADS"] = f'{n_threads}'
os.environ["MKL_NUM_THREADS"] = f'{n_threads}'
os.environ["NUMEXPR_NUM_THREADS"] = f'{n_threads}'
from tl2cgen.contrib.util import _libext
import pathlib
import tl2cgen
import treelite
import joblib
import numpy as np
import skmap_bindings as sb
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
import treelite.sklearn
import warnings
import numpy as np
import time

nthread = 96
# RANDOM FOREST

MODEL_PATH = '/mnt/slurm/jobs/global_soc'
OUT_PATH = 'com_models'
version = '20250122'
prop = 'ocd'

input_path=f'{MODEL_PATH}/model_rf.{prop}_production_v{version}.joblib'
covs_path=f'{MODEL_PATH}/model_rf.{prop}_production.tl2cgen.intel_v{version}.covs'
output_path=f'{MODEL_PATH}/model_rf.{prop}_production_v{version}.so'
output_path_tl2cgen=f'{OUT_PATH}/model_rf.{prop}_production.tl2cgen_v{version}.so'
output_path_tl2cgen_intel=f'{OUT_PATH}/model_rf.{prop}_production.tl2cgen.intel_v{version}.so'

model_rf = joblib.load(input_path)
treelite_model = treelite.sklearn.import_model(model_rf)
with open(covs_path, "w") as file:
    for item in list(model_rf.feature_names_in_):
        file.write(item + "\n")

In [6]:
output_path_tl2cgen_intel

'com_models/model_rf.ocd_production.tl2cgen.intel_v20250122.so'

In [ ]:
tl2cgen.export_lib(treelite_model, toolchain="gcc", libpath=output_path_tl2cgen, nthread = nthread, verbose=True,    params={"parallel_comp": nthread})

In [8]:
tl2cgen.export_lib(treelite_model,
                   libpath = OUT_PATH,
                   nthread = nthread,
                   verbose=True,
                   toolchain= '/opt/intel/oneapi/compiler/2024.0/bin/icx-cc -O3 -xCORE-AVX512 -qopt-zmm-usage=high',
                   params={
                        "verbose": 1,
                        "quantize": 1,
                        "parallel_comp": 96,
                   })

/home/opengeohub/.local/lib/python3.8/site-packages/treelite/core.py:25: UserWarning: [15:53:43] /mnt/ripley/global_soc/scikit-map/tl2cgen/build/_deps/treelite-src/src/serializer.cc:202: The model you are loading originated from a newer Treelite version; some functionalities may be unavailable.
Currently running Treelite version 4.1.2
The model checkpoint was generated from Treelite version 4.5.0
  warnings.warn(py_str(msg))


[15:53:44] /mnt/ripley/global_soc/scikit-map/tl2cgen/src/compiler/ast/split.cc:35: Parallel compilation enabled; member trees will be divided into 96 translation units.
Compiling sources files in directory /tmp/tmp7wtykh_s into object files (*.o)...
Generating dynamic shared library /tmp/tmp7wtykh_s/predictor.so...
Generated shared library in 217.69 seconds


AttributeError: 'PosixPath' object has no attribute 'rstrip'

In [ ]:
# treelite_model.export_lib(
#     toolchain= '/opt/intel/oneapi/compiler/2024.0/bin/icx-cc -O3 -xCORE-AVX512 -qopt-zmm-usage=high',
#     libpath=output_path,
#     params={
#         "verbose": 1,
#         "quantize": 1,
#         "parallel_comp": 96,
#         "code_folding_req": 64., # tune this
#     }
# )


In [3]:
data = np.load("data30m.npy")
# data = np.load("data240m.npy")

if str(treelite_model.input_type) != str(data.dtype):
    if data.dtype == np.float64:
        data_cast = np.empty(data.shape, dtype='float32')
        sb.castFloat64ToFloat32(data, n_threads, data_cast)
    if data.dtype == np.float32:
        data_cast = np.empty(data.shape, dtype='float64')
        sb.castFloat32ToFloat64(data, n_threads, data_cast)
else:
    data_cast = data

predictor = treelite_runtime.Predictor(output_path, nthread=nthread)
predictor2  = tl2cgen.Predictor(output_path_tl2cgen_intel, nthread=nthread)
predictor3  = tl2cgen.Predictor(output_path_tl2cgen, nthread=nthread)

In [4]:

start = time.time()
data_dm = treelite_runtime.DMatrix(data, dtype='float32')
print(f"@@@@@@@@@@@@@@ TreeLite DMatrix: {(time.time() - start):.2f} s")
start1 = time.time()
rest1 = predictor.predict(data_dm)
print(f"@@@@@@@@@@@@@@ TreeLite Predict: {(time.time() - start1):.2f} s")
print(f"@@@@@@@@@@@@@@@@@@@@@@@@@@@ TreeLite: {(time.time() - start):.2f} s")

start = time.time()
dmat = tl2cgen.DMatrix(data, dtype='float32')
print(f"XXXXXXXXXXXXXX tl2cgen DMatrix: {(time.time() - start):.2f} s")
start1 = time.time()
# rest2 = predictor2.predict(dmat)
rest2 = predictor3.predict(dmat)
if rest2.shape[-1] == 1:
    rest2 = np.squeeze(rest2, axis=-1)
if rest2.shape[-1] == 1:
    rest2 = np.squeeze(rest2, axis=-1)
print(f"XXXXXXXXX tl2cgen Predict: {(time.time() - start1):.2f} s")
print(f"XXXXXXXXXXXXXXXXXXX tl2cgen: {(time.time() - start):.2f} s")

start = time.time()
rest3 = model_rf.predict(data, )
print(f"oooooooooooooo Sklearn: {(time.time() - start):.2f} s")

# start = time.time()
# data_float64 = np.empty(data.shape, dtype='float64')
# sb.castFloat32ToFloat64(data, nthread, data_float64)
# rest = treelite.gtil.predict_per_tree(model=treelite_model, data=data_cast, nthread=nthread)
# if rest.shape[-1] == 1:
#     rest = np.squeeze(rest, axis=-1)
# print(f"############################ Per tree manual casting: {(time.time() - start):.2f} s")


# rest.shape

@@@@@@@@@@@@@@ TreeLite DMatrix: 13.08 s
@@@@@@@@@@@@@@ TreeLite Predict: 8.36 s
@@@@@@@@@@@@@@@@@@@@@@@@@@@ TreeLite: 21.44 s
XXXXXXXXXXXXXX tl2cgen DMatrix: 1.04 s
XXXXXXXXX tl2cgen Predict: 7.17 s
XXXXXXXXXXXXXXXXXXX tl2cgen: 8.20 s


/home/opengeohub/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


oooooooooooooo Sklearn: 21.44 s


In [5]:
rest3

array([3.40355592, 3.40355592, 3.40355592, ..., 3.03323275, 3.03323275,
       3.03323275])

In [6]:
rest2

array([3.40355592, 3.40355592, 3.40355592, ..., 3.03323275, 3.03323275,
       3.03323275])

In [7]:
rest1

array([3.40355592, 3.40355592, 3.40355592, ..., 3.03323275, 3.03323275,
       3.03323275])

In [8]:
np.linalg.norm(rest3-rest2)/np.linalg.norm(rest3)

2.38649309331379e-16

In [9]:
np.linalg.norm(rest3-rest1)/np.linalg.norm(rest3)

3.2069207157551277e-16

In [14]:
if "8":
    print("MISO")

MISO


In [15]:
warning("mioai")

NameError: name 'warning' is not defined

In [19]:
40%8.1

7.600000000000001

In [21]:
def wowo(a:str = None, 
                 b:int = None):
    if a:
        print("miao")

In [24]:
wowo()